In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
from vllm import LLM, SamplingParams


/home/yilong/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-03 21:36:10 __init__.py:190] Automatically detected platform cuda.


2025-03-03 21:36:11,109	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom/checkpoint-563")
llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-773")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-300")

INFO 03-03 21:36:14 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 03-03 21:36:14 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-773', speculative_config=None, tokenizer='outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-773', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=outp

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.39it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.39it/s]


INFO 03-03 21:36:15 model_runner.py:1115] Loading model weights took 0.9277 GB


INFO 03-03 21:36:16 worker.py:267] Memory profiling takes 0.45 seconds
INFO 03-03 21:36:16 worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.90) = 21.59GiB
INFO 03-03 21:36:16 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 19.14GiB.
INFO 03-03 21:36:16 executor_base.py:110] # CUDA blocks: 104553, # CPU blocks: 21845
INFO 03-03 21:36:16 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 51.05x
INFO 03-03 21:36:16 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:07<00:00,  4.95it/s]

INFO 03-03 21:36:23 model_runner.py:1562] Graph capturing finished in 7 secs, took 0.48 GiB
INFO 03-03 21:36:23 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 8.06 seconds


In [4]:
sampling_params = SamplingParams(temperature=0.6, max_tokens=512, top_p = 0.95)

# read data

In [15]:
from datasets import load_dataset, Dataset
import re
from tqdm import tqdm
import json

In [16]:
# uncomment middle messages for 1-shot prompting
def get_questions() -> Dataset:
    data = load_dataset('parquet', data_files='data/cleaned_train.parquet')['train']
    test_data = load_dataset('parquet', data_files='data/cleaned_test.parquet')['train']
    return data , test_data


dataset, testset = get_questions()
#testset = get_gsm8k_questions('test')




def extract_xml_answer(text: str) -> str:
    answer_pattern = r'<answer>(.*?)</answer>'
    matches = list(re.finditer(answer_pattern, text, re.DOTALL))
    if not matches:
        return None
        
    final_answer = matches[-1].group(1).strip()
    return final_answer



def normalize_answer(answer: str) -> str:
    """Normalizes the answer text for better comparison.
    Args:
        answer: Raw answer text
    Returns:
        Normalized answer text
    """
    # Convert to lowercase
    normalized = answer.lower()
    # Remove extra whitespace
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    # Remove punctuation that doesn't affect meaning
    normalized = re.sub(r'[.,;:!?]', '', normalized)
    return normalized


In [17]:
def reward_func(response, answer):
    response_ = extract_xml_answer(response)
    norm_response = normalize_answer(response_)
    norm_answer = normalize_answer(answer)
    if norm_response == norm_answer:
        return 1
    else:
        return 0

In [18]:
dataset, testset = get_questions()

In [8]:
i = 0
outputs= llm.chat([testset['prompt'][0], testset['prompt'][1], testset['prompt'][2]], sampling_params)
print(outputs[0].outputs[0].text)
print(testset['answer'][0])

print(outputs[1].outputs[0].text)
print(testset['answer'][1])

print(outputs[2].outputs[0].text)
print(testset['answer'][2])

INFO 03-03 21:04:33 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  8.36it/s, est. speed input: 3183.49 toks/s, output: 209.07 toks/s]

<think> In the toolbox within the center's premises. </think> <answer> toolbox </answer>
plastic bin
<think> In the high-rise embassy building, where the conference room is located. </think> <answer> high-rise embassy building </answer>
conference room
<think> In the consulate meeting room </think> <answer> consulate meeting room </answer>
consulate meeting room


In [19]:
def main_func(llm, testset, n, n_repeat=5):
    # Initialize result storage
    all_results = []
    if type(testset)==pd.core.frame.DataFrame:
        testset.index = range(len(testset))
    # Iterate through test set in chunks of n
    with tqdm(range(0, len(testset['prompt']), n), desc="Processing batches") as pbar:
        for i in pbar:
    #for i in tqdm(range(0, len(testset['prompt']), n), desc="Processing batches"):
            # Extract current batch of prompts and answers
            batch_prompts = [p for p in testset['prompt'][i:i+n]]
            batch_answers = [a for a in testset['answer'][i:i+n]]

            # Get outputs from the LLM for the current batch
            for k in range(n_repeat):
                try:
                    outputs = llm.chat(batch_prompts, 
                                    sampling_params=sampling_params, 
                                    use_tqdm=False)

                    # Check correctness for the batch
                    is_correct = [reward_func(each.outputs[0].text, answer) for each, answer in zip(outputs, batch_answers)]
                    # Add detailed results for each item in the batch
                    for j in range(len(batch_prompts)):
                        result_entry = {
                            "prompt": batch_prompts[j],
                            "answer": batch_answers[j],
                            "output": outputs[j].outputs[0].text,  # Assuming this is how to get the output
                            "correctness": is_correct[j]
                        }
                        all_results.append(result_entry)
                except:
                    print(i, i+n)
                    # for each in batch_prompts:
                    #     print(each)
                    #correct=[each['correctness'] for each in all_results]
                    #print(f"Correct: {np.mean(np.array(correct).flatten())}")
                    if k == n_repeat-1:
                        return all_results
                break
            
            if i%5 == 0:
                correct=[each['correctness'] for each in all_results]
                pbar.set_postfix(accuracy=np.mean(np.array(correct).flatten()))

    # Save results to JSON file
    #save_results(all_results)
    correct=[each['correctness'] for each in all_results]
    print(f"Correct: {np.mean(np.array(correct).flatten())}")
    return all_results

def save_results(results, filename="data/results/evaluation_tom.json"):
    """Saves the evaluation results to a JSON file."""
    with open(filename, mode='w') as file:
        json.dump(results, file, indent=4)
    print(f"Results saved to {filename}")

**384 prompts model**

Processing batches: 100%|██████████| 381/381 [00:54<00:00,  7.04it/s, accuracy=0.679]

Correct: 0.6792152704135737

**512 prompts model 773 check**

Processing batches: 100%|██████████| 381/381 [00:47<00:00,  7.99it/s, accuracy=0.729]

Correct: 0.7290899526564966

**512 prompts model 300 check**

Processing batches: 100%|██████████| 381/381 [00:51<00:00,  7.38it/s, accuracy=0.706]

Correct: 0.7056962025316456

# Alll Test

In [13]:
test_data_all = load_dataset('parquet', data_files='data/test_data_all.parquet')['train']
train_data_512 = load_dataset('parquet', data_files='data/cleaned_train_512.parquet')['train']

In [14]:
train_data_512['prompt'][0]

[{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
  'role': 'system'},
 {'content': "Story: The consulate meeting room was a quiet, formal space with a large wooden table and leather chairs, surrounded by walls lined with the flags of different countries. Soft morning light filtered in through the tall windows, casting a warm glow on the metal file cabinet in the corner. Peyton pushed open the door, and the soft creak of the hinge broke the silence as she stepped into the consulate meeting room. The silver letter opener came to rest on the top of the metal file cabinet, its normally mundane presence now part of the carefully managed arrangement of objects in the consulate meeting room. A faint s

**384 model**

Processing batches: 100%|██████████| 533/533 [01:21<00:00,  6.54it/s, accuracy=0.632]

Correct: 0.6317606444188723

**512 token 773**

Processing batches: 100%|██████████| 533/533 [01:10<00:00,  7.57it/s, accuracy=0.685]

Correct: 0.6851990984222389

**512 token 300**

Processing batches: 100%|██████████| 533/533 [01:15<00:00,  7.02it/s, accuracy=0.66] 

Correct: 0.6595022624434389

In [15]:
results_all = main_func(llm, test_data_all, 5, n_repeat=10)

Processing batches:  73%|███████▎  | 390/533 [00:55<00:15,  9.02it/s, accuracy=0.659]

1940 1945


Processing batches:  95%|█████████▌| 509/533 [01:12<00:04,  5.79it/s, accuracy=0.658]

2535 2540


Processing batches: 100%|██████████| 533/533 [01:15<00:00,  7.02it/s, accuracy=0.66] 

Correct: 0.6595022624434389


# Other Set

In [26]:
test_faux = load_dataset('parquet', data_files='data/test/test_faux.parquet')['train']
test_hitom = load_dataset('parquet', data_files='data/test/test_hitom.parquet')['train']
test_tomi = load_dataset('parquet', data_files='data/test/test_tomi.parquet')['train']
test_hitom_fixed = load_dataset('parquet', data_files='data/test/test_hitom_fixed.parquet')['train']

Generating train split: 600 examples [00:00, 118846.87 examples/s]


In [33]:
test_hitom

Dataset({
    features: ['deception', 'story_length', 'question_order', 'sample_id', 'story', 'question', 'choices', 'answer', 'question_old', 'answer_old', 'prompt'],
    num_rows: 600
})

In [34]:
test_hitom['prompt'][0]

[{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
  'role': 'system'},
 {'content': 'Story: Avery, Charlotte, Isabella, Elizabeth and Owen entered the living_room.\nThe lettuce is in the green_drawer.\nAvery moved the lettuce to the green_bathtub.\nElizabeth dislikes the tangerine.\nAvery exited the living_room.\nCharlotte moved the lettuce to the blue_pantry.\nOwen likes the green_envelope.\nCharlotte exited the living_room.\nIsabella made no movements and stayed in the living_room for 1 minute.\nIsabella exited the living_room.\nOwen dislikes the green_pepper.\nElizabeth made no movements and stayed in the living_room for 1 minute.\nElizabeth exited the living_room.\nOwen moved the lettuce to 

In [35]:
outputs= llm.chat([test_hitom['prompt'][0], test_hitom['prompt'][1], test_hitom['prompt'][2]], sampling_params)
print(outputs[0].outputs[0].text)
print(test_hitom['answer'][0])

print(outputs[1].outputs[0].text)
print(test_hitom['answer'][1])

print(outputs[2].outputs[0].text)
print(test_hitom['answer'][2])

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 3/3 [00:00<00:00, 10.03it/s, est. speed input: 2370.28 toks/s, output: 250.73 toks/s]

<think> The lettuce is in the green_drawer. </think> <answer> green_drawer </answer>
green_drawer
<think> Jacob lost his watch. </think> <answer> waiting_room </answer>
blue_container
<think> After the sequence of events, the eggplant is in the green_envelope. </think> <answer> green_envelope </answer>
green_envelope


In [36]:
results_hitom = main_func(llm, test_hitom, 5, n_repeat=3)

Processing batches:   0%|          | 0/120 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 120/120 [00:18<00:00,  6.66it/s, accuracy=0.313]

Correct: 0.31333333333333335


In [38]:
results_tomi = main_func(llm, test_tomi[90:100], 5, n_repeat=3)

Processing batches:   0%|          | 0/2 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 2/2 [00:00<00:00,  3.07it/s, accuracy=0]

Correct: 0.0


In [40]:
results_tomi

[{'prompt': [{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
    'role': 'system'},
   {'content': 'Story: Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.\n Question:Where was the broccoli at the beginning?',
    'role': 'user'}],
  'answer': 'box',
  'output': '<think>Jackson and Chloe entered the hall. </think> <answer> dining_room </answer>',
  'correctness': 0},
 {'prompt': [{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are 

In [39]:
test_tomi[90:100]

{'story': ['Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.',
  'Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.',
  'Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.',
  'Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.',
  'Jackson entered the hall.\nChloe entered the hall.\nThe boots is in the bathtub.\nJackson exited the hall.\nJackson entered the dining_room.\nChloe moved the boots to the pantry.',
  'Jackson entered the hall.\nChloe entered the hall.\nThe boots is in 